In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
dataset = load_dataset('csv', data_files='dataset_name.csv', split='train')
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
!pip install transformers
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert_model_name')

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples['text'], padding="max_length", truncation=True)

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained('bert_model_name', num_labels=2)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, auc
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    auc = roc_auc_score(labels, logits[:, 1])
    fpr, tpr, _ = roc_curve(labels, logits[:, 1])
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions),
        "auc": auc,
        "roc": (fpr, tpr),
        "confusion_matrix": {
            "tn": tn,
            "fp": fp,
            "fn": fn,
            "tp": tp
        }
    }

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import KFold

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", num_train_epochs=5)

kf = KFold(n_splits=10, shuffle=True)

eval_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(tokenized_data["train"])):
    print("Training fold {}...".format(fold+1))

    train_dataset = tokenized_data["train"].select(train_idx)
    val_dataset = tokenized_data["train"].select(val_idx)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_result = trainer.evaluate(eval_dataset=val_dataset)
    eval_results.append(eval_result)

In [ ]:
accuracies = []
f1_scores = []
auc_scores = []
confusion_matrices = []

for eval_result in eval_results:
    accuracies.append(eval_result["eval_accuracy"])
    f1_scores.append(eval_result["eval_f1"])
    auc_scores.append(eval_result["eval_auc"])
    confusion_matrices.append(eval_result["eval_confusion_matrix"])

avg_accuracy = sum(accuracies) / len(accuracies)
avg_f1_score = sum(f1_scores) / len(f1_scores)
avg_auc_score = sum(auc_scores) / len(auc_scores)

total_confusion_matrix = {'tn': 0, 'fp': 0, 'fn': 0, 'tp': 0}

for confusion_matrix in confusion_matrices:
    total_confusion_matrix['tn'] += confusion_matrix['tn']
    total_confusion_matrix['fp'] += confusion_matrix['fp']
    total_confusion_matrix['fn'] += confusion_matrix['fn']
    total_confusion_matrix['tp'] += confusion_matrix['tp']

num_folds = len(confusion_matrices)
avg_confusion_matrix = {
    'tn': total_confusion_matrix['tn'] // num_folds,
    'fp': total_confusion_matrix['fp'] // num_folds,
    'fn': total_confusion_matrix['fn'] // num_folds,
    'tp': total_confusion_matrix['tp'] // num_folds
}

print("Average evaluation results over all folds:")
print("Accuracy:", avg_accuracy)
print("F1 score:", avg_f1_score)
print("AUC score:", avg_auc_score)
print("Confusion matrix:", avg_confusion_matrix)

In [ ]:
model_name = 'my_model'
trainer.save_model(model_name)